In [12]:
import sys
dir = '/scratch/programming/pipeline/preprocessing-pipeline/'
sys.path.append(dir+'in_development/Will')
sys.path.append(dir+'src')
from cell_extractor.CellDetectorTrainer import CellDetectorTrainer
from cell_extractor.DetectorMetrics import DetectorMetricsDK55
import pandas as pd
import os

In [6]:
trainer = CellDetectorTrainer('DK55',round=2,segmentation_threshold=2300)
trainer.list_detectors()

{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}


['detector_round_2_threshold_2300.pkl',
 'detector_round_2_threshold_2200.pkl',
 'detector_round_2_threshold_2100.pkl',
 'detector_round_2.pkl',
 'detector_round_2_threshold_2000.pkl',
 'detector_round_1.pkl',
 'detector_round_2_threshold_2700.pkl']

In [7]:
all_detections = []
for threshold in [2000,2100,2200,2300,2700]:
    trainer = CellDetectorTrainer('DK55',round=2,segmentation_threshold=threshold)
    detections = trainer.load_detections()
    detections = detections[['row','col','section','predictions']]
    detections['name'] = str(threshold)
    all_detections.append(detections)
all_detections = pd.concat(all_detections)

{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}
{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}
{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}
{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}
{'eta': 0.3, 'objective': 'binary:logistic', 'nthread': 7}


In [8]:
sures = all_detections[all_detections.predictions==2]
unsures = all_detections[all_detections.predictions==0]

In [9]:
sures = sures.drop('predictions',axis=1)
unsures = unsures.drop('predictions',axis=1)
sures.columns = ['x','y','section','name']
unsures.columns = ['x','y','section','name']

In [17]:
sures.head()

,x,y,section,name
0,18851,37633,132,2000
1,17423,38133,145,2000
2,17638,37839,145,2000
3,17708,37725,145,2000
4,17794,37634,145,2000


In [19]:
from abakit.lib.SqlController import SqlController
import pandas as pd
import numpy as np
controller = SqlController('DK55')
for _,celli in sures.iterrows():
    coord = np.array([celli.y,celli.x,celli.section])*np.array([0.325,0.325,20])
    controller.add_layer_data_row('DK55',34,1,coord,52,'detected_soma_multi_level')


In [14]:
util = DetectorMetricsDK55()
sure_path = os.path.join(util.qc_annotation_input_path,'mutithreshold_sure.csv')
unsure_path = os.path.join(util.qc_annotation_input_path,'mutithreshold_sure.csv')
sures.to_csv(sure_path)
unsures.to_csv(unsure_path)

In [ ]:
util = DetectorMetricsDK55(sure_file_name = sure_path,unsure_file_name=unsure_path)

In [15]:
util.calculate_qualification()

calculating distance matrix
finding points that are close to each other
grouping and labeling points that are close to each other
before removing duplicates 16813
after removing duplicates 14108
computer missed, human detected
299
computer sure, human negative
50
computer  UNsure, human negative
247
computer UNsure, human positive
402
Total computer UNsure
1285
computer UNsure, human unmarked
634
computer sure, human unmarked
10560
More than 2 labels
27
Computer Detected, Human Missed
10560
total train
1883
Human mind change
32
original training set after mind change
1851
